# Check fo duplicate articles and data objects
A list of publications is obtainded from the app database. This list will contain a titles, IDs and DOIs which need to be explored to look for duplicates. 
The steps of the process are: 
 1. get a Title, DOI, and URL for each publication
 2. revise each element of the list for duplicates

In [1]:
# Libraries
# library containign functions that read and write to csv files
import lib.handle_csv as csvh
# library for connecting to the db
import lib.handle_db as dbh
# library for handling text matchings
import lib.text_comp as txtc
# library for getting data from crossref
import lib.crossref_api as cr_api
# library for handling url searchs
import lib.handle_urls as urlh
# managing files and file paths
from pathlib import Path
# add aprogress bar
from tqdm import tqdm_notebook 
from tqdm import tqdm
#library for handling json files
import json
# library for using regular expressions
import re
# library for handling http requests
import requests
# import custom functions (common to various notebooks)
import processing_functions as pr_fns

# datetime parsing
from datetime import datetime

current_step = 3

## Verify if there are duplicates in articles

1. Open the current publication list from the appdb
2. read each entry and check if there are duplicates in doi, url or title



In [2]:
# deal with no pdf_file column
def get_pubs_list(db_path):
    pubs_list = None
    try:
        pubs_list = pr_fns.get_pub_data(db_path)
    except Exception as inst:
        if 'pdf_file' in inst.args[0]:
            print('problem articles table does not have pdf_file column')
            pass
    try:
        if pubs_list == None:
            pubs_list = pr_fns.get_pub_app_data(db_path)
    except Exception as inst:
        print(type(inst))
        print(inst.args)
        print(inst)
        print('another problem')    
    return pubs_list

# 1 current app DB
db_name = 'development'
#ukchapp_db = "db_files/"+db_name+".sqlite3"
ukchapp_db = "../mcc_data/"+db_name+".sqlite3"
while not Path(ukchapp_db).is_file():
    print('Please enter the name of app db file:')
    ukchapp_db = input()
    
#  get publication data from the ukch app
app_pubs = get_pubs_list(ukchapp_db)

problem articles table does not have pdf_file column


In [3]:
    
# 2 read each entry and check if there are duplicates in doi, url or title
dup_list={}
dup_count = 0
if current_step == 1 and app_pubs != None:  
    dups = []
    for idx, a_pub in enumerate(tqdm_notebook(app_pubs)):
        pub_id = a_pub[0]
        pub_title = a_pub[1]
        pub_doi = a_pub[2]
        pub_url = a_pub[3]
        # verfy if dois are duplicated
        if pub_doi != None:
            for i_indx in range(idx+1, len(app_pubs)):
                #print(pub_doi, app_pubs[i_indx][2])
                if app_pubs[i_indx][2]!=None and pub_doi.strip().lower() ==  app_pubs[i_indx][2].strip().lower():
                    print("DOI", pub_doi, "duplicated at:", i_indx, app_pubs[i_indx], app_pubs[idx] )
                    a_dup = {'pub_comp': app_pubs[idx], 'pub_dup': app_pubs[i_indx],"dup_at":'DOI'}
                    dup_count+=1
                    dup_list[dup_count] = a_dup
        # verify if urls are all unique
        if pub_url != None:
            for i_indx in range(idx+1, len(app_pubs)):
                if app_pubs[i_indx][3]!=None and pub_url.strip().lower() ==  app_pubs[i_indx][3].strip().lower():
                    print("URL", pub_url, "duplicated at:", i_indx, app_pubs[i_indx], app_pubs[idx] )    
                    a_dup = {'pub_comp': app_pubs[idx], 'pub_dup': app_pubs[i_indx],"dup_at":'URL'}
                    dup_count+=1
                    dup_list[dup_count] = a_dup
        # verify if titles are all unique
        if pub_title != None:
            for i_indx in range(idx+1, len(app_pubs)):
                similarity = txtc.similar(pub_title.strip().lower(), app_pubs[i_indx][1].strip().lower())
                #print(similarity)
                if app_pubs[i_indx][1]!=None and pub_title.strip().lower() ==  app_pubs[i_indx][1].strip().lower():
                    print("Title", pub_title, "duplicated at:", i_indx,app_pubs[i_indx][1], app_pubs[idx][1])     
                    print("Similarity:", similarity)
                    a_dup = {'pub_comp': app_pubs[idx], 'pub_dup': app_pubs[i_indx],"dup_at":'Title'}
                    dup_count+=1
                    dup_list[dup_count] = a_dup
                #elif similarity > 0.8:
                #    print(similarity, "Title:\n\t", pub_title, "\n\t- similar at:\n\t", i_indx,app_pubs[i_indx][1]) 
if len(dup_list) > 0:
        csvh.write_csv_data(dup_list, 'dup_'+db_name+'.csv')
else:
    print ("No duplicate articles in DB")

No duplicate articles in DB


## Verify if there are duplicates in data objects

1. get the current data objects list from the appdb
2. read each entry and check if there are duplicates in doi, url or title



In [4]:
# 1 current app DB
dup_list={}
db_name = 'development'
#ukchapp_db = "db_files/"+db_name+".sqlite3"
ukchapp_db = "../mcc_data/"+db_name+".sqlite3"
while not Path(ukchapp_db).is_file():
    print('Please enter the name of app db file:')
    ukchapp_db = input()
    
#  get datasets list from the ukch app
app_datasetes = pr_fns.get_dataset_data(ukchapp_db)

# 2 read each entry and check if there are duplicates in doi, url or title
if current_step == 2:  
    dup_list={}
    dup_count = 0
    for idx, a_ds in enumerate(tqdm_notebook(app_datasetes)):
        ds_id = a_ds[0]
        ds_doi = a_ds[1]
        ds_url = a_ds[2]
        ds_name = a_ds[3]
        #print (ds_id, ds_doi, ds_url, ds_name)
        # verfy if dois are duplicated
        if ds_doi != None and ds_doi != '':
            for i_indx in range(idx+1, len(app_datasetes)):
                #print(pub_doi, app_pubs[i_indx][1])
                if app_datasetes[i_indx][1]!=None and ds_doi.strip().lower() ==  app_datasetes[i_indx][1].strip().lower():
                    print("DOI", ds_doi, "duplicated at:", i_indx, app_datasetes[i_indx], app_datasetes[idx] )
                    a_dup = {'pub_comp': app_datasetes[idx], 'pub_dup': app_datasetes[i_indx],"dup_at":'DOI'}
                    dup_count+=1
                    dup_list[dup_count] = a_dup
        # verify if urls are all unique
        if ds_url != None:
            for i_indx in range(idx+1, len(app_datasetes)):
                #print(app_datasetes[i_indx])
                if app_datasetes[i_indx][2]!=None and ds_url.strip().lower() ==  app_datasetes[i_indx][2].strip().lower():
                    print("URL", ds_url, "duplicated at:", i_indx, app_datasetes[i_indx], app_datasetes[idx] )  
                    a_dup = {'pub_comp': app_datasetes[idx], 'pub_dup': app_datasetes[i_indx],"dup_at":'URL'}
                    dup_count+=1
                    dup_list[dup_count] = a_dup
        # verify if titles are all unique
        if ds_name != None:
            for i_indx in range(idx+1, len(app_datasetes)):
                similarity = txtc.similar(ds_name.strip().lower(), app_datasetes[i_indx][3].strip().lower())
                #print(similarity)
                if app_datasetes[i_indx][1]!=None and ds_name.strip().lower() ==  app_datasetes[i_indx][3].strip().lower():
                    print("Title", ds_name, "duplicated at:", i_indx,app_datasetes[i_indx][3], app_datasetes[idx][3])     
                    print("Similarity:", similarity)
                    a_dup = {'pub_comp': app_datasetes[idx], 'pub_dup': app_datasetes[i_indx],"dup_at":'Title'}
                    dup_count+=1
                    dup_list[dup_count] = a_dup
                #elif similarity > 0.8:
                #    print(similarity, "Title:\n\t", ds_name, "\n\t- similar at:\n\t", i_indx,app_datasetes[i_indx][3]) 
                #    break
if len(dup_list) > 0:
    csvh.write_csv_data(dup_list, 'dup_do_'+db_name+'.csv')
else:
    print ("No duplicate DOs in DB")

No duplicate DOs in DB


# Verify authors

1. verify that there are no authors with no articles in the DB
2. verify that authors are unique remove close matches (need to check spellings)

In [5]:
def make_like_str(a_string):
    like_str = "%" + re.sub(r'[^a-zA-Z\s:]', '%', a_string) + "%"
    return like_str

def get_all_authors():
    db_conn = dbh.DataBaseAdapter(ukchapp_db)
    s_table = 'authors'
    s_fields = 'id, given_name, last_name, orcid'
    s_where = 'isap IS NULL' # 1 displayed authors - 0/NULL the rest
    authors_list = db_conn.get_values(s_table, s_fields, s_where)
    return authors_list

def get_null_authors():
    db_conn = dbh.DataBaseAdapter(ukchapp_db)
    s_table = 'authors'
    s_fields = 'id, given_name, last_name, orcid'
    s_where = 'id NOT IN (SELECT article_authors.author_id FROM article_authors)'
    authors_list = db_conn.get_values(s_table, s_fields, s_where)
    return authors_list

def get_similar_authors(name,surname,orcid):
    db_conn = dbh.DataBaseAdapter(ukchapp_db)
    s_table = 'authors'
    s_fields = 'id, given_name, last_name, orcid'
    like_surname = make_like_str(surname)
    surname.replace("'","''")
    s_where = "(orcid = '%s' AND last_name = '%s')"%(orcid, surname)
    s_where += "OR(given_name = '%s' AND last_name = '%s')"%(name, surname)
    s_where += "OR(last_name = '%s')"%(surname)
    s_where += "OR(last_name LIKE '%s')"%(like_surname)
    authors_list = db_conn.get_values(s_table, s_fields, s_where)
    return authors_list

def count_linked(author_id):
    db_conn = dbh.DataBaseAdapter(ukchapp_db)
    s_table = 'article_authors'
    s_fields = 'id, author_id'
    s_where = "(author_id = %s)"%(author_id)
    aa_list = db_conn.get_values(s_table, s_fields, s_where)
    return len(aa_list)

In [6]:
from IPython.display import clear_output

def save_ok_list(values_list, file_name):
    with open(file_name, 'w') as f:
        for an_id in values_list:
            f.write(str(an_id)+'\n')

def open_ok_list(file_name):
    with open(file_name) as f:
        lines = f.readlines()
    from_file = []
    for a_line in lines:
        from_file.append(int(a_line.replace('\n','')))
    return from_file

def add_to_ok_list(a_value, file_name):
    with open(file_name, 'a') as f:
        f.write(str(a_value)+'\n')

In [7]:
def is_single_word(a_word, another_word):
    single_word = False
    in_word = another_word.lower().find(a_word.lower())
    if in_word >= 0:
        single_word = True
        if in_word > 0 and  another_word[in_word-1].isalpha():
            sinlge_word = False
        if in_word + len(a_word) < len(another_word)-1 and another_word[in_word + len(a_word)].isalpha():
            single_word = False
    return single_word

def prune_similar_surnames(the_similars, a_surname):
    pruned_list = []
    for a_simi in the_similars:
        if a_simi[2] == a_surname or is_single_word(a_surname,a_simi[2]) :
            pruned_list.append(a_simi)
    return pruned_list

def get_initials(given_names):
    initials_1 =  [a_letter for a_letter in given_names if a_letter.isupper() ] 
    names = given_names.split()
    initials_2 = [a_name[0] for a_name in given_names]
    ri_1 = ". ".join(initials_1)+"."
    ri_2 = " ".join(initials_1)
    return ri_1, ri_2
    
def prune_similar_names(the_similars, a_name):
    pruned_list = []
    dot_initials, initials = get_initials(a_name)
    for a_simi in the_similars:
        if a_simi[1] == a_name or is_single_word(a_name,a_simi[1]) :
            pruned_list.append(a_simi)
        elif initials == a_simi[1] or dot_initials == a_simi[1]:
            pruned_list.append(a_simi)    
    return pruned_list

In [8]:
def set_author_value(a_id, a_column, a_value):
    db_conn = dbh.DataBaseAdapter(ukchapp_db)
    db_conn.set_value_table('authors', a_id, a_column, a_value)


def update_author(old_author, a_id, a_name, a_surname, a_orcid):
    if a_id != old_author[0]:
        return # do not update different authors        
    if a_name != old_author[1]:
        set_author_value(a_id, 'given_name', a_name)
    if a_surname != old_author[2]:
        set_author_value(a_id, 'last_name', a_surname)
    if a_orcid != old_author[3]:
        set_author_value(a_id, 'given_name', a_orcid)

def update_article_authors(new_id, old_id):
    db_conn = dbh.DataBaseAdapter(ukchapp_db)
    s_where = "author_id = %s" % (old_id)
    aa_ids = db_conn.get_values('article_authors', 'id', s_where)
    print(aa_ids)
    for an_id in aa_ids:
        db_conn.set_value_table('article_authors', an_id[0], 'author_id', new_id)

def delete_author(a_id):
    db_conn = dbh.DataBaseAdapter(ukchapp_db)
    db_conn.connection.execute("DELETE FROM authors WHERE id = %s" % (a_id ))
    db_conn.connection.commit()
    
def merge_authors(an_author, auth_similars):
    auth_id = an_author[0]
    auth_name = an_author[1]
    auth_surname = an_author[2]
    auth_orcid = an_author[3]

    for a_result in auth_similars:
        if auth_id > a_result[0]:
            auth_id = a_result[0]
        if auth_name != a_result[1] and len(auth_name) < len(a_result[1]):
            print("Which name \n\t 1)", auth_name, "\n\t 2)", a_result[1])
            opt_name = input()
            if opt_name == '2': auth_name = a_result[1] 
        if auth_surname != a_result[2] and len(auth_surname) < len(a_result[2]):
            print("Which surname \n\t 1)", auth_surname, "\n\t 2)", a_result[2])
            opt_name = input()
            if opt_name == '2': auth_surname = a_result[2] 
        if auth_orcid != a_result[3]:
            print("Which ORCID \n\t 1)", auth_orcid, "\n\t 2)", a_result[3])
            opt_name = input()
            if opt_name == '2': auth_orcid = a_result[3]

    if auth_id != an_author[0] or auth_name != an_author[1] or auth_surname != an_author[2] or auth_orcid != an_author[3]:
        print ("Will update", an_author, "to", auth_id , auth_name , auth_surname,auth_orcid )

    for a_result in auth_similars:
        if auth_id != a_result[0]:
            print("will update all author articles from:", a_result[0], "to:", auth_id)
            print("will delete author:", a_result[0])
    print("Continue?\n\t 1) proceed \n\t 2) cancel")
    opt_go = input()
    if opt_go == '1':
        update_author(an_author, auth_id, auth_name , auth_surname,auth_orcid)
        for a_result in auth_similars:
            if auth_id != a_result[0]:
                update_article_authors(auth_id,a_result[0])
                delete_author(a_result[0])


## Check authors with no articles

In [9]:
# verify that there are no authors with no articles in the DB
no_artaut_authors = get_null_authors()

delete_these=[]
for an_author in no_artaut_authors:
    dup_id = an_author[0]
    dup_name = an_author[1]
    dup_surname = an_author[2]
    dup_orcid = an_author[3] if an_author[3]!=None else "NULL"
    print(dup_id, dup_name, dup_surname, dup_orcid)
    similars = get_similar_authors(dup_name, dup_surname, dup_orcid)
    print('There are %s similar authors in DB'%(len(similars)))
    for idx, a_simil in enumerate(similars):
        art_count = count_linked(a_simil[0])
        if art_count == 0:
            print(idx, a_simil,"Links:", art_count, "DELETE")
            delete_these.append(a_simil[0])
        else:
            print(idx, a_simil,"Links:", art_count, "CHECK")

print(delete_these)

[]


## Review duplicate authors (by name and last name)

In [10]:
# Manually review probable duplicate authors 
all_authors = get_all_authors()
revise_these=[]

safe_list = open_ok_list('safe_list.txt')
pacer_idx = 0
for an_author in all_authors:   
    dup_id = an_author[0]
    dup_name = an_author[1]
    dup_surname = an_author[2]
    dup_orcid = an_author[3] if an_author[3]!=None else "NULL"
    if not(int(dup_id) in safe_list):
        if "'" in dup_surname: dup_surname = dup_surname.replace("'","''")
        if ("’") in dup_surname: dup_surname = dup_surname.replace("’","''")
        all_similars = get_similar_authors(dup_name, dup_surname, dup_orcid)
        similars = prune_similar_surnames(all_similars, dup_surname)
        similars = prune_similar_names(similars, dup_name)
        if len(similars) > 1 and  len(dup_surname)>3 :
            print("*************************************************")
            print("Author:", dup_id, dup_name, dup_surname, dup_orcid)
            print('There are %s similar authors in DB'%(len(similars)))

            for idx, a_simil in enumerate(similars):
                art_count = count_linked(a_simil[0])
                print(idx, a_simil,"Links:", art_count)
            print ("Options:\n\t (1) Ignore \n\t (2) Merge\n\t (3) next")
            sel_action = input()
            if sel_action == '1':
                safe_list = sorted(list(set(safe_list + [sublist[0] for sublist in similars])))
            if sel_action == '2':
                merge_authors(an_author, similars)
            pacer_idx+=1
            clear_output()
            if pacer_idx == 10:
                break
print("OK:", len(safe_list))
print(safe_list)
save_ok_list(safe_list, 'safe_list.txt')



OK: 34
[1, 147, 158, 166, 179, 202, 395, 550, 557, 572, 664, 921, 956, 1036, 1086, 1172, 1269, 1939, 1969, 1974, 2010, 2400, 2721, 2738, 2855, 2870, 2965, 3012, 3104, 3181, 3231, 3248, 3344, 3375]


### Verify affiliations and author affiliations agains crossref affiliations
1 get group of crossref affiliations
2 get assigned affiliation
3 verify if OK if not show and ask for action


In [11]:
import craffiparser

def get_parser(db_):
    cr_parse = craffiparser.crp(db_)
    cr_parse.start_lists()
    return cr_parse

def get_cr_affis_article_author_ids(db_name):
    db_conn = dbh.DataBaseAdapter(db_name)
    a_table = 'cr_affiliations'
    a_column = 'article_author_id'
    cr_affis_article_author_ids = db_conn.get_value_list(a_table, a_column)
    return cr_affis_article_author_ids

def get_cr_lines_for_article_author_ids(db_name, art_author_id):
    db_conn = dbh.DataBaseAdapter(db_name)
    s_table = 'cr_affiliations'
    s_fields = '*'
    s_where = "article_author_id = %s"%(art_author_id)
    authors_list = db_conn.get_values(s_table, s_fields, s_where)
    return authors_list

def get_affiliation_id(db_name, parsed_affi):
    db_conn = dbh.DataBaseAdapter(db_name)
    s_table = 'affiliations'
    s_field = 'id'
    for k,v in parsed_affi.items():
        if "'" in v :parsed_affi[k]=v.replace("'","''")
    
    list_where = [ k +" = '"+ v +"'" for k,v in parsed_affi.items() if k != 'address']
    s_where = " AND ".join(list_where) 
    s_where = s_where.replace("= ''", "IS NULL")
    #print (s_where)
    affi_list = db_conn.get_values(s_table, s_field, s_where)
    affi_id = None
    if affi_list !=[]:
        affi_id = affi_list[0][0]
    return affi_id

def get_auth_affi_affiliation_id(db_name, aut_affi_id):
    db_conn = dbh.DataBaseAdapter(db_name)
    s_table = 'author_affiliations'
    s_field = 'affiliation_id'
    s_where = " id = %i" %(aut_affi_id)
    #print (s_where)
    affi_list = db_conn.get_values(s_table, s_field, s_where)
    if affi_list !=[]:
        affi_list = list(set([an_id[0] for an_id in affi_list]))
    return affi_list

def is_one_line_affi(cr_parser, str_affi):
    is_one_liner = False
    parsed_affi = cr_parser.split_single(str_affi)
    parsed_no_blanks = {k:v for k,v in parsed_affi.items() if v != ''}
    if len(parsed_no_blanks) > 1:
        is_one_liner = True
    return is_one_liner

def check_assigned_affi_ol(db_name, cr_parser, cr_affi):
    assigned_ok = False
    if cr_affi[3] != None:
        parsed_affi = cr_parser.split_single(cr_affi[1])
        parsed_no_blanks = {k:v for k,v in parsed_affi.items() if v != ''}
        affi_id = get_affiliation_id(db_name, parsed_affi)
        aut_affi = get_auth_affi_affiliation_id(db_name, cr_affi[3])
        if aut_affi[0] == affi_id:
            print('Assigned ID:', aut_affi[0], "Recoverd ID:", affi_id)
            assigned_ok = True
    return assigned_ok

def parse_multiline(db_name, cr_parser, cr_affis):
    print('multiline affiliation')
    print('try to join multiline')
    affi_key = list(parsed_no_blanks.keys())[0]
    if parsed_multiline =={}:
        parsed_multiline = parsed_affi
    elif parsed_multiline[affi_key] =="":
        parsed_multiline[affi_key] = parsed_no_blanks[affi_key]
    elif parsed_multiline[affi_key] !="" and affi_key == 'address':
        parsed_multiline[affi_key] += ", " + parsed_no_blanks[affi_key]
    elif len (auth_cr_affis) == len(multiline_no_blanks):# or parsed_multiline[affi_key] !="":
        # check if OK, and start new collector 
        affi_id = get_parsed_affiliation_id(parsed_multiline)
        aut_affi = get_auth_affi_affiliation_id(current_affi)
        if affi_id in aut_affi:
            # assigned id matches calculated affi_id
            print("assigned id:", aut_affi, "matches calculated:", affi_id)
            affis_ok = True
            parsed_multiline
        else:
            print("Author:", art_aut_id, "Collected: ", len (auth_cr_affis))
            print("assigned id:", aut_affi, "does not match calculated:", affi_id)
            affis_ok = False
            break
    parsed_multiline = parsed_affi

# Problems:
#   Mismatch in assigned affiliation
#   Affiliation not assigned

# 1 Get list of article_author_ids from CR_affi
# 2 For each article_author_id:
#   1 Get CR_affi lines
#   2 verify CR_affi lines
#     1 check if 
#        a) one affiliation per cr_affi 
#           parse each single affiliation
#           check if assigned affiliation is OK (assigned ID matches calculated ID)
#        b) multiple lines form an affiliation (2+)
#           parse each multi-line affiliation
#           check if assigned affiliation is OK

affi_parser = get_parser(ukchapp_db)

list_art_aut_ids = get_cr_affis_article_author_ids(ukchapp_db)

for art_aut_id in list_art_aut_ids:
    print ('Article Author: ', art_aut_id)
    cr_lines = get_cr_lines_for_article_author_ids(ukchapp_db, art_aut_id)
    print(cr_lines)
    all_one_liners = True
    for a_cr_line in cr_lines:
        one_line_affi = is_one_line_affi(affi_parser, a_cr_line[1])
        print( a_cr_line[1], one_line_affi)
        if not one_line_affi:
            all_one_liners = False
    if all_one_liners:
        print('verify one liners')
        for a_cr_line in cr_lines:
            assigned_ok = check_assigned_affi_ol(ukchapp_db, affi_parser, a_cr_line)
            if not assigned_ok:
                print("Problems with ", a_cr_line)
    else:
        print('verify multiline affi')
        break


Refreshing lists
Article Author:  1
[(1, 'Chemistry─School of Natural and Environmental Sciences, Newcastle University, Newcastle upon Tyne, NE1 7RU, U.K.', 1, 1, '2022-08-23 08:21:07.822289', '2022-08-28 18:40:22.195960'), (2, 'Centre for Energy, Newcastle University, Newcastle upon Tyne, NE1 7RU, U.K.', 1, 2, '2022-08-23 08:21:07.832527', '2022-08-28 18:40:22.218108')]
Chemistry─School of Natural and Environmental Sciences, Newcastle University, Newcastle upon Tyne, NE1 7RU, U.K. True
Centre for Energy, Newcastle University, Newcastle upon Tyne, NE1 7RU, U.K. True
verify one liners
Assigned ID: 1 Recoverd ID: 1
Assigned ID: 2 Recoverd ID: 2
Article Author:  2
[(3, 'Department of Chemistry, University of Bath, Bath, BA2 7AY, U.K.', 2, 7, '2022-08-23 08:21:07.882489', '2022-08-28 18:44:26.111714'), (4, 'Department of Materials, University of Oxford, Oxford, OX1 3PH, U.K.', 2, 8, '2022-08-23 08:21:07.894823', '2022-08-28 18:44:26.131624')]
Department of Chemistry, University of Bath, Ba

verify one liners
Assigned ID: 442 Recoverd ID: 442
Article Author:  78
[(27, 'School of Science and Technology, Nottingham Trent University, Clifton Lane, NG11 8NS Nottingham, United Kingdom', 78, 2211, '2022-08-24 11:33:37.913368', '2022-08-24 11:33:37.913368')]
School of Science and Technology, Nottingham Trent University, Clifton Lane, NG11 8NS Nottingham, United Kingdom True
verify one liners
Assigned ID: 442 Recoverd ID: 442
Article Author:  79
[(28, 'Department of Chemical Sciences, Indian Institute of Science Education and Research (IISER) Mohali, Sector 81, Knowledge City, Manauli 140306, India', 79, 3157, '2022-08-24 11:33:37.955240', '2022-08-24 11:33:37.955240')]
Department of Chemical Sciences, Indian Institute of Science Education and Research (IISER) Mohali, Sector 81, Knowledge City, Manauli 140306, India True
verify one liners
Assigned ID: 441 Recoverd ID: 441
Article Author:  80
[(29, 'School of Science and Technology', 80, 3516, '2022-08-24 11:50:08.873479', '2022-08

In [18]:
cr_lines = get_cr_lines_for_article_author_ids(ukchapp_db, 1210)
print(cr_lines)

[(1834, 'School of Chemistry', 1210, 469, '2022-08-24 11:52:10.755049', '2022-08-28 21:07:58.823088'), (1835, 'University of Leeds', 1210, 469, '2022-08-24 11:52:10.762317', '2022-08-28 21:07:58.833093'), (1836, 'Leeds LS2 9JT', 1210, 469, '2022-08-24 11:52:10.772644', '2022-08-28 21:07:58.841728'), (1837, 'UK', 1210, 469, '2022-08-24 11:52:10.780377', '2022-08-28 21:07:58.858861'), (1838, 'School of Chemistry', 1210, -1, '2022-08-24 11:52:10.788217', '2022-08-24 11:52:10.788217')]


In [ ]:
import craffiparser

def get_cr_affiliations(db_name):
    db_conn = dbh.DataBaseAdapter(db_name)
    cr_affis = db_conn.get_full_table('cr_affiliations')
    return cr_affis

def get_parsed_affiliation_id(parsed_affi):
    db_conn = dbh.DataBaseAdapter(ukchapp_db)
    s_table = 'affiliations'
    s_field = 'id'
    for k,v in parsed_affi.items():
        if "'" in v :parsed_affi[k]=v.replace("'","''")
    
    list_where = [ k +" = '"+ v +"'" for k,v in parsed_affi.items() if k != 'address']
    s_where = " AND ".join(list_where) 
    s_where = s_where.replace("= ''", "IS NULL")
    #print (s_where)
    affi_list = db_conn.get_values(s_table, s_field, s_where)
    affi_id = None
    if affi_list !=[]:
        affi_id = affi_list[0][0]
    return affi_id

def get_auth_affi_affiliation_id(aut_affi_id):
    db_conn = dbh.DataBaseAdapter(ukchapp_db)
    s_table = 'author_affiliations'
    s_field = 'affiliation_id'
    s_where = " id = %i" %(aut_affi_id)
    #print (s_where)
    affi_list = db_conn.get_values(s_table, s_field, s_where)
    if affi_list !=[]:
        affi_list = list(set([an_id[0] for an_id in affi_list]))
    return affi_list

def set_auth_affi_affiliation_id(aut_affi_id, affi_id):
    db_conn = dbh.DataBaseAdapter(ukchapp_db)
    s_table = 'author_affiliations'
    s_field = 'affiliation_id'
    db_conn = dbh.DataBaseAdapter(ukchapp_db)
    db_conn.set_value_table(s_table, aut_affi_id,  s_field , affi_id)


def start_parser(db_):
    cr_parse = craffiparser.crp(db_)
    cr_parse.start_lists()
    return cr_parse

def check_art_author_affi(cr_parse, art_aut_id, auth_cr_affis): 
    error_code = 0
    current_affi = 0
    parsed_multiline ={}
    for an_affi in auth_cr_affis:
        if current_affi == 0 and an_affi[3]!=None:
            current_affi = an_affi[3]
        elif current_affi!= an_affi[3] and an_affi[3]!=None:
            current_affi = an_affi[3]
        elif an_affi[3]==None:
            print("Affiliation not assigned")
            print(an_affi)
            error_code = 3
            break
        parsed_affi = cr_parse.split_single(an_affi[1])
        parsed_no_blanks = {k:v for k,v in parsed_affi.items() if v != ''}
        if len(parsed_no_blanks)>1:
            affi_id = get_parsed_affiliation_id(parsed_affi)
            aut_affi = get_auth_affi_affiliation_id(current_affi)
            #print(affi_id)
            if affi_id in aut_affi:
                #print("assigned id:", aut_affi, "matches calculated:", affi_id)
                error_code = 0
            else:
                print("assigned id:", aut_affi, "does not match calculated:", affi_id)
                if affi_id == None:
                    error_code = 2
                else:
                    error_code = 1
                break
        if len(parsed_no_blanks)==1:
            error_code = 4
            print("Could be multiline", parsed_no_blanks, error_code)
    return error_code

def add_affiliation(affi_values):
    db_conn = dbh.DataBaseAdapter(ukchapp_db)
    add_update_time = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    affiliation_new = affi_values
    del affiliation_new['address']
    if 'address' in affiliation_new.keys():
        del affiliation_new['address']
    if 'num' in affiliation_new.keys():
        del affiliation_new['num']
    affiliation_new['created_at'] = add_update_time
    affiliation_new['updated_at'] = add_update_time
    affiliation_id = db_conn.put_values_table("affiliations", affiliation_new.keys(), affiliation_new.values())
    return affiliation_id

def add_address(addr_str, affi_id, ctry_str):
    db_conn = dbh.DataBaseAdapter(ukchapp_db)
    add_update_time = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    affi_address = {'add_01':addr_str,"affiliation_id":affi_id,'country':ctry_str,
                    'created_at': add_update_time, 'updated_at':add_update_time}
    add_id = db_conn.put_values_table("addresses", affi_address.keys(), affi_address.values())
    return add_id

def institution_manual_entry(affiliation_values):
    print("*********** enter institution ************************")
    print("Provide institution for:", affiliation_values)            
    ins_str = input()
    affiliation_values['institution'] = ins_str
    return affiliation_values


def custom_split(cr_parse, affiliation_values):
    # Look for institution name should be in the address element
    print ("Select how to parse ",  affiliation_values)
    option_how = 0
    while not option_how in [1,2,3]:
        print('options: ')
        print('1: parse address element')
        print('2: enter manually')
        print('3: do nothing ')
        print('selection:')
        str_opt = input()
        if str_opt != '':
            option_how = int(str_opt)
        else:
            option_how = 3
    if option_how == 1:
        affiliation_values = cr_parse.manual_split_address(affiliation_values)
    elif option_how == 2:
        affiliation_values = institution_manual_entry(affiliation_values)
    return affiliation_values


def add_missing(cr_parse, auth_cr_affis):
    current_affi = 0
    for an_affi in auth_cr_affis:
        if current_affi == 0 and an_affi[3]!=None:
            current_affi = an_affi[3]
        elif current_affi!= an_affi[3] and an_affi[3]!=None:
            current_affi = an_affi[3]
        elif an_affi[3]==None:
            print("Affiliation not assigned")
            print(an_affi)
            error_code = 3
            break
        parsed_affi = cr_parse.split_single(an_affi[1])
        print("* Affi Values:", parsed_affi)
        affiliation_id = address_id = 0
        # determine if parsing needs help (institution found?)   
        if parsed_affi['institution'] in ["", None]:
            parsed_affi = custom_split(cr_parse, parsed_affi)
        else: 
            print('could not process string:', auth_cr_affis)
            print('results obtained:', parsed_affi)
            
        address_string = parsed_affi['address']
        affiliation_id = add_affiliation(parsed_affi)
        # renew lists as new affi is added
        cr_parse.start_lists()
        address_id = add_address(address_string, affiliation_id, parsed_affi['country'])
        
        print ('* Added affiliation and address', affiliation_id, address_id)    
        print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')

def correct_mistmatch_affi(cr_parse, auth_cr_affis):
    current_affi = 0
    for an_affi in auth_cr_affis:
        if current_affi == 0 and an_affi[3]!=None:
            current_affi = an_affi[3]
        elif current_affi!= an_affi[3] and an_affi[3]!=None:
            current_affi = an_affi[3]
        elif an_affi[3]==None:
            print("Affiliation not assigned")
            print(an_affi)
            error_code = 3
            break
        parsed_affi = cr_parse.split_single(an_affi[1])
        parsed_no_blanks = {k:v for k,v in parsed_affi.items() if v != ''}
        if len(parsed_no_blanks)>1:
            affi_id = get_parsed_affiliation_id(parsed_affi)
            aut_affi = get_auth_affi_affiliation_id(current_affi)
            #print(affi_id)
            if not affi_id in aut_affi and len(aut_affi) == 1:
                print("assigned id:", aut_affi, "does not match calculated:", affi_id)
                set_auth_affi_affiliation_id(current_affi, affi_id)
                input()
            else:
                print("assigned id:", aut_affi, "matches calculated:", affi_id)
                input()

affi_error={1:"Missmatch assignement", 2:"Affiliation not found", 
            3:"Affiliation not assigned", 4:"Multiline affiliation"}
parser_obj = start_parser(ukchapp_db)

cr_affi_data = get_cr_affiliations(ukchapp_db)
print("Verifying", len(cr_affi_data), "Afiliations")
current_art_author = 0
collected_current = []
problem_assignements=[]

safe_list = open_ok_list('ok_affi_list.txt')
for a_cr_line in cr_affi_data:
    if current_art_author == 0:
        current_art_author = int(a_cr_line[2])
    if current_art_author != int(a_cr_line[2]) and not current_art_author in safe_list:
        # change of author verify if saved affis are OK
        verify_result = check_art_author_affi (parser_obj, current_art_author, collected_current)
        if not verify_result in [0,4]:
            clear_output()
            print("problems with author affi ", collected_current[0][3], "**", affi_error[verify_result])
            problem_assignements.append(current_art_author)
            if a_cr_line[3] != None: aut_affi = get_auth_affi_affiliation_id(a_cr_line[3])  
            else: aut_affi = 0
            print("****************** PARSED **************************")
            print(collected_current)
            print("****************** ASSIGNED **************************")
            print(aut_affi)
            print("Options:\n\t1) Fix\n\t2) Ignore\n\t3) End" )
            option_af =""
            option_af=input()
            if option_af == '1':
                if verify_result in [2]:
                    add_missing(parser_obj, collected_current)
                if verify_result == 1:
                    correct_mistmatch_affi(parser_obj, collected_current)
            if option_af=='2':
                safe_list.append(current_art_author)
            elif option_af=='3':
                break
        elif verify_result == 0:
            safe_list.append(current_art_author)
        current_art_author = int(a_cr_line[2])
        collected_current = [a_cr_line]
    else:
        # same author keep collecting affi names
        collected_current.append(a_cr_line)
    
print("OK:", len(safe_list))
print(safe_list)
save_ok_list(safe_list, 'ok_affi_list.txt')
 

In [ ]:
option_af

In [ ]:
###############################################################################################################################
# Check later

import craffiparser

def get_cr_affiliations(db_name):
    db_conn = dbh.DataBaseAdapter(db_name)
    cr_affis = db_conn.get_full_table('cr_affiliations')
    return cr_affis

def get_parsed_affiliation_id(parsed_affi):
    db_conn = dbh.DataBaseAdapter(ukchapp_db)
    s_table = 'affiliations'
    s_field = 'id'
    for k,v in parsed_affi.items():
        if "'" in v :parsed_affi[k]=v.replace("'","''")
    
    list_where = [ k +" = '"+ v +"'" for k,v in parsed_affi.items() if k != 'address']
    s_where = " AND ".join(list_where) 
    s_where = s_where.replace("= ''", "IS NULL")
    #print (s_where)
    affi_list = db_conn.get_values(s_table, s_field, s_where)
    affi_id = None
    if affi_list !=[]:
        affi_id = affi_list[0][0]
    return affi_id

def get_auth_affi_affiliation_id(aut_affi_id):
    db_conn = dbh.DataBaseAdapter(ukchapp_db)
    s_table = 'author_affiliations'
    s_field = 'affiliation_id'
    s_where = " id = %i" %(aut_affi_id)
    #print (s_where)
    affi_list = db_conn.get_values(s_table, s_field, s_where)
    if affi_list !=[]:
        affi_list = list(set([an_id[0] for an_id in affi_list]))
    return affi_list

def set_auth_affi_affiliation_id(aut_affi_id, affi_id):
    db_conn = dbh.DataBaseAdapter(ukchapp_db)
    s_table = 'author_affiliations'
    s_field = 'affiliation_id'
    db_conn = dbh.DataBaseAdapter(ukchapp_db)
    db_conn.set_value_table(s_table, aut_affi_id,  s_field , affi_id)


def start_parser(db_):
    cr_parse = craffiparser.crp(db_)
    cr_parse.start_lists()
    return cr_parse

def check_art_author_affi(cr_parse, art_aut_id, auth_cr_affis): 
    error_code = 0
    current_affi = 0
    parsed_multiline ={}
    for an_affi in auth_cr_affis:
        if current_affi == 0 and an_affi[3]!=None:
            current_affi = an_affi[3]
        elif current_affi!= an_affi[3] and an_affi[3]!=None:
            current_affi = an_affi[3]
        elif an_affi[3]==None:
            print("Affiliation not assigned")
            print(an_affi)
            error_code = 3
            break
        parsed_affi = cr_parse.split_single(an_affi[1])
        parsed_no_blanks = {k:v for k,v in parsed_affi.items() if v != ''}
        
        if len(parsed_no_blanks)>1:
            affi_id = get_parsed_affiliation_id(parsed_affi)
            aut_affi = get_auth_affi_affiliation_id(current_affi)
            #print(affi_id)
            if affi_id in aut_affi:
                #print("assigned id:", aut_affi, "matches calculated:", affi_id)
                error_code = 0
            else:
                #clear_output()
       #         print("Author:", art_aut_id, "Collected: ", len (auth_cr_affis))
                print("assigned id:", aut_affi, "does not match calculated:", affi_id)
       #         print("****************** PARSED **************************")
       #         print(parsed_affi)
       #         print("****************** ASSIGNED **************************")
       #         print(an_affi)
       #         print("1) fix 2) Ignore")
       #         option_af=input()
                
       #         affis_ok = False
                if affi_id == None:
                    error_code = 2
                else:
                    error_code = 1
            
                break
        if len(parsed_no_blanks)==1:
        #    print('multiline affiliation')
        #    print('try to join multiline')
        #    affi_key = list(parsed_no_blanks.keys())[0]
        #    if parsed_multiline =={}:
        #        parsed_multiline = parsed_affi
        #    elif parsed_multiline[affi_key] =="":
        #        parsed_multiline[affi_key] = parsed_no_blanks[affi_key]
        #    elif parsed_multiline[affi_key] !="" and affi_key == 'address':
        #        parsed_multiline[affi_key] += ", " + parsed_no_blanks[affi_key]
        #    elif len (auth_cr_affis) == len(multiline_no_blanks):# or parsed_multiline[affi_key] !="":
        #        # check if OK, and start new collector 
        #        affi_id = get_parsed_affiliation_id(parsed_multiline)
        #        aut_affi = get_auth_affi_affiliation_id(current_affi)
        #        if affi_id in aut_affi:
        #            # assigned id matches calculated affi_id
        #            print("assigned id:", aut_affi, "matches calculated:", affi_id)
        #            affis_ok = True
        #            parsed_multiline
        #        else:
        #            print("Author:", art_aut_id, "Collected: ", len (auth_cr_affis))
        #            print("assigned id:", aut_affi, "does not match calculated:", affi_id)
        #            affis_ok = False
        #            break
        #    parsed_multiline = parsed_affi
        #    multiline_no_blanks = {k:v for k,v in parsed_multiline.items() if v != ''}    
        #    if len (auth_cr_affis) == len(multiline_no_blanks):
        #        affi_id = get_parsed_affiliation_id(parsed_multiline)
        #        aut_affi = get_auth_affi_affiliation_id(current_affi)
        #        if affi_id in aut_affi:
        #            # assigned id matches calculated affi_id
        #            print("assigned id:", aut_affi, "matches calculated:", affi_id)
        #            affis_ok = True
        #            parsed_multiline
        #        else:
        #            print("Author:", art_aut_id, "Collected: ", len (auth_cr_affis))
        #            print("assigned id:", aut_affi, "does not match calculated:", affi_id)
        #            affis_ok = False
        #            break
            print("Could be multiline", parsed_no_blanks, error_code)
    return error_code

def add_affiliation(affi_values):
    db_conn = dbh.DataBaseAdapter(ukchapp_db)
    add_update_time = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    affiliation_new = affi_values
    del affiliation_new['address']
    if 'address' in affiliation_new.keys():
        del affiliation_new['address']
    if 'num' in affiliation_new.keys():
        del affiliation_new['num']
    affiliation_new['created_at'] = add_update_time
    affiliation_new['updated_at'] = add_update_time
    affiliation_id = db_conn.put_values_table("affiliations", affiliation_new.keys(), affiliation_new.values())
    return affiliation_id

def add_address(addr_str, affi_id, ctry_str):
    db_conn = dbh.DataBaseAdapter(ukchapp_db)
    add_update_time = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    affi_address = {'add_01':addr_str,"affiliation_id":affi_id,'country':ctry_str,
                    'created_at': add_update_time, 'updated_at':add_update_time}
    add_id = db_conn.put_values_table("addresses", affi_address.keys(), affi_address.values())
    return add_id

def institution_manual_entry(affiliation_values):
    print("*********** enter institution ************************")
    print("Provide institution for:", affiliation_values)            
    ins_str = input()
    affiliation_values['institution'] = ins_str
    return affiliation_values


def custom_split(cr_parse, affiliation_values):
    # Look for institution name should be in the address element
    print ("Select how to parse ",  affiliation_values)
    option_how = 0
    while not option_how in [1,2,3]:
        print('options: ')
        print('1: parse address element')
        print('2: enter manually')
        print('3: do nothing ')
        print('selection:')
        str_opt = input()
        if str_opt != '':
            option_how = int(str_opt)
        else:
            option_how = 3
    if option_how == 1:
        affiliation_values = cr_parse.manual_split_address(affiliation_values)
    elif option_how == 2:
        affiliation_values = institution_manual_entry(affiliation_values)
    return affiliation_values


def add_missing(cr_parse, auth_cr_affis):
    current_affi = 0
    for an_affi in auth_cr_affis:
        if current_affi == 0 and an_affi[3]!=None:
            current_affi = an_affi[3]
        elif current_affi!= an_affi[3] and an_affi[3]!=None:
            current_affi = an_affi[3]
        elif an_affi[3]==None:
            print("Affiliation not assigned")
            print(an_affi)
            error_code = 3
            break
        parsed_affi = cr_parse.split_single(an_affi[1])
        print("* Affi Values:", parsed_affi)
        affiliation_id = address_id = 0
        # determine if parsing needs help (institution found?)   
        if parsed_affi['institution'] in ["", None]:
            parsed_affi = custom_split(cr_parse, parsed_affi)
        else: 
            print('could not process string:', auth_cr_affis)
            print('results obtained:', parsed_affi)
            
        address_string = parsed_affi['address']
        affiliation_id = add_affiliation(parsed_affi)
        # renew lists as new affi is added
        cr_parse.start_lists()
        address_id = add_address(address_string, affiliation_id, parsed_affi['country'])
        
        print ('* Added affiliation and address', affiliation_id, address_id)    
        print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')

def correct_mistmatch_affi(cr_parse, auth_cr_affis):
    current_affi = 0
    for an_affi in auth_cr_affis:
        if current_affi == 0 and an_affi[3]!=None:
            current_affi = an_affi[3]
        elif current_affi!= an_affi[3] and an_affi[3]!=None:
            current_affi = an_affi[3]
        elif an_affi[3]==None:
            print("Affiliation not assigned")
            print(an_affi)
            error_code = 3
            break
        parsed_affi = cr_parse.split_single(an_affi[1])
        parsed_no_blanks = {k:v for k,v in parsed_affi.items() if v != ''}
        if len(parsed_no_blanks)>1:
            affi_id = get_parsed_affiliation_id(parsed_affi)
            aut_affi = get_auth_affi_affiliation_id(current_affi)
            #print(affi_id)
            if not affi_id in aut_affi and len(aut_affi) == 1:
                print("assigned id:", aut_affi, "does not match calculated:", affi_id)
                set_auth_affi_affiliation_id(current_affi, affi_id)
                input()
            else:
                print("assigned id:", aut_affi, "matches calculated:", affi_id)
                input()

affi_error={1:"Missmatch assignement", 2:"Affiliation not found", 
            3:"Affiliation not assigned", 4:"Multiline affiliation"}
parser_obj = start_parser(ukchapp_db)

cr_affi_data = get_cr_affiliations(ukchapp_db)
print("Verifying", len(cr_affi_data), "Afiliations")
current_art_author = 0
collected_current = []
problem_assignements=[]

safe_list = open_ok_list('ok_affi_list.txt')
for a_cr_line in cr_affi_data:
    if current_art_author == 0:
        current_art_author = int(a_cr_line[2])
    if current_art_author != int(a_cr_line[2]):
        # change of author verify if saved affis are OK
        verify_result = check_art_author_affi (parser_obj, current_art_author, collected_current)
        if verify_result !=0:
            clear_output()
            print("problems with author affi ", collected_current[0][3], "**", affi_error[verify_result])
            problem_assignements.append(current_art_author)
            if a_cr_line[3] != None: aut_affi = get_auth_affi_affiliation_id(a_cr_line[3])  
            else: aut_affi = 0
            print("****************** PARSED **************************")
            print(collected_current)
            print("****************** ASSIGNED **************************")
            print(aut_affi)
            print("Options:\n\t1) Fix\n\t2) Ignore\n\t3) End" )
            option_af=input()
            if option_af == '1':
                if verify_result in [2]:
                    add_missing(parser_obj, collected_current)
                if verify_result == 1:
                    correct_mistmatch_affi(parser_obj, collected_current)
            if option_af=='2':
                add_to_ok_list(current_art_author, 'ok_affi_list.txt')
            elif option_af=='3':
                break
        else:
            add_to_ok_list(current_art_author, 'ok_affi_list.txt')
        current_art_author = int(a_cr_line[2])
        collected_current = [a_cr_line]
    else:
        # same author keep collecting affi names
        collected_current.append(a_cr_line)
    
    #if current_art_author > 26:
    #    break   

## Check that pdf files exist 

Use the data on the articles table to verify if file are stored in the corresponding folder
We also check that the files in the folder are all accounted for (have a corersponding record)

In [ ]:
if current_step == 2:
    # get publication data from the ukch app
    app_pubs = pr_fns.get_pub_data(ukchapp_db)

    dups = []
    for idx, a_pub in enumerate(tqdm_notebook(app_pubs)):
        pub_id = a_pub[0]
        pub_title = a_pub[1]
        pub_doi = a_pub[2]
        pub_url = a_pub[3]
        for i_indx in range(idx,len(app_pubs)):
            if not (pub_doi is None) and pub_doi.strip().lower() ==  app_pubs[i_indx][2]:
                print(pub_doi, "duplicated at:", i_indx) 

In [ ]:
ukchapp_db

In [ ]:
if current_step == 2:
    for infile in tqdm_notebook(Path("pdf_files").glob('*.pdf')):
        file_found = False
        for a_pub in app_pubs:
            if infile.name == a_pub[4]:
                file_found = True
                break
        if not file_found:
            print("Not in DB:", infile.name, "DB Name", a_pub[4])

## Get missing pdfs

In [ ]:
# use regular expression to check if a given string
# is a valid DOI, using pattern from CR
def valid_doi(cr_doi):
    # CR DOIS: https://www.crossref.org/blog/dois-and-matching-regular-expressions/
    # CR DOIs re1
    # /^10.\d{4,9}/[-._;()/:A-Z0-9]+$/i
    if cr_doi == None:
        return False
    cr_re_01 = '^10.\d{4,9}/[-._;()/:A-Z0-9]+'
    compare = re.match(cr_re_01, cr_doi, re.IGNORECASE)
    if compare != None and cr_doi == compare.group():
        return True
    else:
        return False
# get publication data from the ukch app
db_pubs = pr_fns.get_pub_data(ukchapp_db)

if current_step == 2:
    for a_pub in tqdm_notebook(db_pubs):
        if a_pub[0] > 616:
            pub_id = a_pub[0]
            pub_title = a_pub[1]
            pub_doi = a_pub[2]
            pub_url = a_pub[3]
            pub_pdf = a_pub[4]
            pub_html = a_pub[5]
            if pub_pdf == None:
                not_in_url = True
                print("ID: ", pub_id, "Publication: ",pub_title,
                      "\n\tDOI: ", pub_doi, " URL: ", pub_url)
                if "pdf" in pub_url:
                    print ("\tTry to get the pdf from URL: ", pub_url)
                    try:
                        response = requests.get(pub_url)
                        content_type = response.headers['content-type']
                        if not 'text' in content_type:
                            #print(response.headers)
                            cd= response.headers['content-disposition']
                            #print(cd)
                            fname = re.findall("filename=(.+)", cd)[0]
                            #print(fname)
                            if not Path('pdf_files/' + pdf_file).is_file():
                                with open('pdf_files/'+ fname +'.pdf', 'wb') as f:
                                    f.write(response.content)
                            else:
                                set_pdf_file_value(pdf_file, pub_id, ukchapp_db)
                            not_in_url = False
                    except:
                        print("ID: ", pub_id, "\nPublication: ",pub_title, 
                               "\nDOI: ", pub_doi, "\nDOI: ", pub_url) 
                if not_in_url:
                    print("\tTry to see if json file has link to pdf: ")
                    if valid_doi(pub_doi):
                        crjd, doi_file = pr_fns.get_cr_json_object(pub_doi)
                        got_pdf = False
                        if "link" in crjd.keys():
                            for a_link in crjd["link"]:
                                if "\tURL" in a_link.keys() and ("pdf" in a_link["URL"] or "pdf" in a_link["content-type"]):
                                    cr_url = a_link["URL"]
                                    #print("URL: ", cr_url)
                                    pdf_file = get_pdf_from_url(cr_url)
                                    # if the name corresponds to a existing file, assign value to db_record
                                    if Path('pdf_files/' + pdf_file).is_file():
                                        print("\tFile name:", pdf_file)
                                        set_pdf_file_value(pdf_file, pub_id, ukchapp_db)
                                        got_pdf = True
                                    else:
                                        print("\tcould not get file from", cr_url)
                        else: 
                            print("\tno links in json", pub_doi)
                    if not got_pdf and "elsevier" in pub_url:
                        print("\tTrying elsevier doi:" )
                        pdf_file = pr_fns.get_elsevier_pdf(pub_doi)
                        if Path('pdf_files/' + pdf_file).is_file():
                            print("\tFile name:", pdf_file)
                            pr_fns.set_pdf_file_value(pdf_file, pub_id, ukchapp_db)
                            got_pdf = True
                    elif not got_pdf and "wiley" in pub_url:
                        print("\tTrying wiley doi:" )
                        pdf_file = pr_fns.get_wiley_pdf(pub_doi)
                        if Path('pdf_files/' + pdf_file).is_file():
                            print("\tFile name:", pdf_file)
                            pr_fns.set_pdf_file_value(pdf_file, pub_id, ukchapp_db)
                            got_pdf = True
                    elif not got_pdf and "pubs.acs" in pub_url:
                        print("\tTrying acs doi:" )
                        pdf_file = pr_fns.get_acs_pdf(pub_doi)
                        if Path('pdf_files/' + pdf_file).is_file():
                            print("\tFile name:", pdf_file)
                            pr_fns.set_pdf_file_value(pdf_file, pub_id, ukchapp_db)
                            got_pdf = True
                    if not got_pdf:
                        print("\tTry doi:  https://doi.org/" + pub_doi)
    


## Use pdfminer to get metadata from pdf file

In [ ]:
import pdfminer
from pdfminer import high_level as pdfmnr_hl

# functions for PDFminer

def get_pdf_text(pdf_file):
    return pdfmnr_hl.extract_text(pdf_file)

# get the paragraph fragments with references to data
def get_ref_sentences(pdf_text):
    sentences = pdf_text.split("\n")
    groups=[]
    for sentence in sentences:
        if pr_fns.is_data_stmt(sentence.lower()):
            idx = sentences.index(sentence)
            groups.append([idx-1,idx,idx+1])
    reduced_groups = []
    for group in groups:
        idx_group = groups.index(group)
        if groups.index(group) > 0:
            set_g = set(group)
            # make the array before current a set
            set_bg = set(groups[idx_group - 1])
            # make the array after current a set
            set_ag = set()
            if idx_group + 1 < len(groups):    
                set_ag = set(groups[idx_group + 1])
            if len(set_bg.intersection(set_g)) > 0:
                ordered_union = list(set_bg.union(set_g))
                ordered_union.sort()
                reduced_groups.append(ordered_union)
            if len(set_ag.intersection(set_g)) > 0:
                ordered_union = list(set_ag.union(set_g))
                ordered_union.sort()
                reduced_groups.append(ordered_union)
            if len(reduced_groups) > 0:
                is_in_rg = False
                for a_rg in reduced_groups:
                    if set_g.issubset(a_rg):
                        is_in_rg = True
                        break
                if not is_in_rg:
                    reduced_groups.append(list(set_g))
    return_group = []
    for sentence_group in reduced_groups:
        full_sentence = ""
        for single_sentence in sentence_group:
            full_sentence += sentences[single_sentence].strip()
        return_group.append(full_sentence)
    return return_group

# get the paragraph fragments with references to data
def get_all_data_sentences(pdf_text):
    sentences = pdf_text.split("\n")
    groups=[]
    for sentence in sentences:
        if 'data' in sentence.lower() or 'inform' in sentence.lower():
            idx = sentences.index(sentence)
            groups.append([idx-1, idx, idx+1])
    reduced_groups = []
    for group in groups:
        idx_group = groups.index(group)
        if groups.index(group) > 0:
            set_g = set(group)
            # make the array before current a set
            set_bg = set(groups[idx_group - 1])
            # make the array after current a set
            set_ag = set()
            if idx_group + 1 < len(groups):    
                set_ag = set(groups[idx_group + 1])
            if len(set_bg.intersection(set_g)) > 0:
                ordered_union = list(set_bg.union(set_g))
                ordered_union.sort()
                reduced_groups.append(ordered_union)
            if len(set_ag.intersection(set_g)) > 0:
                ordered_union = list(set_ag.union(set_g))
                ordered_union.sort()
                reduced_groups.append(ordered_union)
            if len(reduced_groups) > 0:
                is_in_rg = False
                for a_rg in reduced_groups:
                    if set_g.issubset(a_rg):
                        is_in_rg = True
                        break
                if not is_in_rg:
                    reduced_groups.append(list(set_g))
    return_group = []
    for sentence_group in reduced_groups:
        full_sentence = ""
        for single_sentence in sentence_group:
            full_sentence += sentences[single_sentence].strip()
        if not full_sentence in return_group:
            return_group.append(full_sentence)
    return return_group

# get the http strings from references to data
def get_http_ref(sentence):
    http_frag = ""
    if 'http' in sentence.lower():
        idx_http = sentence.lower().index('http')
        http_frag = sentence[idx_http:]
        space_in_ref = True
        while " " in http_frag:
            space_idx = http_frag.rfind(" ")
            http_frag = http_frag[:space_idx]
        if(http_frag[-1:]=="."):
            http_frag = http_frag[:-1]
    return http_frag

In [ ]:
if current_step == 2:
    # get publication data from the ukch app
    db_pubs = pr_fns.get_pub_data(ukchapp_db)

    # get the list of dois already mined for data 
    input_file = 'pub_data_add202012.csv'
    id_field = 'num'
    processed, headings = csvh.get_csv_data(input_file, id_field)
    for id_num in processed:
        current_title = processed[id_num]['doi']
    processed[1]['num']

    processed_dois = []
    for entry in processed:
        if not processed[entry]['doi'] in processed_dois:
            processed_dois.append( processed[entry]['doi'])

    data_records = {}
    data_mentions = {}
    ref_count = mention_count = 0
    for a_pub in tqdm_notebook(db_pubs):
        data_refs = []
        data_sents = []
        if a_pub[0] > 616:
            pub_id = a_pub[0]
            pub_title = a_pub[1]
            pub_doi = a_pub[2]
            pub_url = a_pub[3]
            pub_pdf = a_pub[4]
            pub_html = a_pub[5]
            if pub_pdf == 'None':
                print("*************************")
                print("Missing PDF for:", pub_doi)
                print("*************************")
            else:
                pdf_file = "pdf_files/" + pub_pdf
                if not Path(pdf_file).is_file():
                    print("*************************")
                    print("Missing file for:", pdf_file, "for", pub_doi)
                    print("*************************")
                else: 
                    print("PDF filename", pdf_file)
                    pdf_text = get_pdf_text(pdf_file)
                    ref_sentences = get_ref_sentences(pdf_text)
                    data_sentences = get_all_data_sentences(pdf_text)
                    for r_sentence in ref_sentences:
                        dt_link = get_http_ref(r_sentence)
                        if 'supplem' in r_sentence.lower():
                            data_refs.append({'type':'supplementary',"desc":r_sentence, 'data_url':dt_link})
                        else:
                            data_refs.append({'type':'supporting',"desc":r_sentence, 'data_url':dt_link})
                    for d_sentence in data_sentences:
                        dt_link = get_http_ref(d_sentence)
                        if 'supplem' in d_sentence.lower():
                            data_sents.append({'type':'supplementary',"desc":d_sentence, 'data_url':dt_link})
                        else:
                            data_sents.append({'type':'supporting',"desc":d_sentence, 'data_url':dt_link})
            if data_refs != []:
                for data_ref in data_refs:
                    data_record = {'id':pub_id, 'doi':pub_doi}    
                    data_record.update(data_ref)
                    data_records[ref_count] = data_record
                    ref_count += 1
            if data_sents != []:
                for data_sent in data_sents:
                    sentence_record = {'id':pub_id, 'doi':pub_doi}    
                    sentence_record.update(data_sent)
                    data_mentions[mention_count] = sentence_record
                    mention_count += 1

Write the results to a csv file to be checked to verify if data mentions are actually 

In [ ]:
#if len(data_records) > 0:
#    csvh.write_csv_data(data_records, 'pdf_data.csv')
if current_step == 2:    
    if len(data_mentions) > 0:
        csvh.write_csv_data(data_mentions, 'pdf_mentions202110.csv')

Verify if the mentions of data or information actually can be linked to data objects

In [ ]:
from IPython.display import clear_output

if current_step == 3:
    print(ukchapp_db)
    print(len(app_pubs))
    # Open results file
    data_mentions, dm_headers = csvh.get_csv_data('pdf_mentions202110.csv')
    print(dm_headers)
    art_id = ''
    for dm in data_mentions:
        if data_mentions[dm]['action']=='':
            clear_output()
            print ("*******************************************")
            print ("Article id  :", data_mentions[dm]['id'])
            print ("DOI         :", data_mentions[dm]['doi'])
            print ("Type        :", data_mentions[dm]['type'], '\tLine:', dm)
            print ("Description :\n\t", data_mentions[dm]['desc'])
            print ("data_url :", data_mentions[dm]['data_url'])
            print ("*******************************************")
            decide_action = False
            while not decide_action:
                print('Action:')
                print('\ta) review')
                print('\tb) none')
                print('\tSelect a or b:')
                lts = input()
                if lts == "a":
                    data_mentions[dm]['action'] = 'review'
                    decide_action = True
                elif lts == "b":
                    data_mentions[dm]['action'] = 'none'
                    decide_action = True
        art__id = data_mentions[dm]['id']
        if dm > 1700:
            break
    if len(data_mentions) > 0:
       csvh.write_csv_data(data_mentions, 'pdf_mentions202110.csv')
    

In [ ]:
# clear the output after each loop cycle
from IPython.display import clear_output

# display editable spreadsheet
import ipysheet


# show gds parameters in a spreadsheet on jupyter
def show_gds(gds_group):
    gds_list = gds_to_list(gds_group)
    #print(gds_list)
    #add 10 more rows in case we need more parameters
    for i in range(10):
        gds_list.append([(len(gds_list)-1)+1,None,None,None,None])
    a_sheet = ipysheet.sheet(rows=len(gds_list), columns=len(gds_list[0]))
    ipysheet.cell_range(gds_list)
    display(a_sheet)
    return a_sheet

if current_step == 3:
    print(ukchapp_db)
    print(len(app_pubs))
    # Open results file
    data_mentions, dm_headers = csvh.get_csv_data('pdf_mentions202110.csv')
    print(dm_headers)
    art_id = ''
    terminate = False
    additional_rows = {}
    for dm in data_mentions:
        if data_mentions[dm]['action']=='review':
            clear_output()
            print ("*******************************************")
            print ("Article id  :", data_mentions[dm]['id'])
            print ("DOI         :", data_mentions[dm]['doi'])
            print ("Type        :", data_mentions[dm]['type'], '\tLine:', dm)
            print ("Description :\n\t", data_mentions[dm]['desc'])
            print ("data_url :", data_mentions[dm]['data_url'])
            print ("*******************************************")
            decide_action = False
            while not decide_action:
                print('Action:')
                print('\ta) review: https://doi.org/'+data_mentions[dm]['doi'])
                print('\ts) add new row')
                print('\td) next')
                print('\tf) terminate')
                print('\tSelect a, s, d, f:')
                lts = input()
                if lts == "a":
                    data_mentions[dm]['action'] = 'reviewed'
                    print ('https://doi.org/'+data_mentions[dm]['doi'])
                    print ('link:',data_mentions[dm]['link'])
                    add_this = input()
                    data_mentions[dm]['link'] = add_this
                    print ('issue:',data_mentions[dm]['issue'])
                    add_this = input()
                    data_mentions[dm]['issue'] = add_this
                    print ('name:',data_mentions[dm]['name'])
                    add_this = input()
                    data_mentions[dm]['name'] = add_this
                    print ('file:',data_mentions[dm]['file'])
                    add_this = input()
                    data_mentions[dm]['file'] = add_this
                if lts == "s":
                    #add a new row
                    new_idx = len(data_mentions) + len(additional_rows) + 1
                    additional_rows[new_idx] = data_mentions[dm]
                    print ('link:',additional_rows[new_idx]['link'])
                    add_this = input()
                    additional_rows[new_idx]['link'] = add_this
                    print ('issue:',additional_rows[new_idx]['issue'])
                    add_this = input()
                    additional_rows[new_idx]['issue'] = add_this
                    print ('name:',additional_rows[new_idx]['name'])
                    add_this = input()
                    additional_rows[new_idx]['name'] = add_this
                    print ('file:',additional_rows[new_idx]['file'])
                    add_this = input()
                    additional_rows[new_idx]['file'] = add_this
                elif lts == "d":
                    if data_mentions[dm]['action'] != 'reviewed':
                        data_mentions[dm]['action'] = 'none'
                    decide_action = True
                elif lts == 'f':
                    decide_action = True
                    terminate = True
        art__id = data_mentions[dm]['id']
        if dm > 1700 or terminate:
            break
    if len(additional_rows)> 0 :
        for nr in additional_rows:
            for a_field in dm_headers:
                data_mentions[nr][a_field] = additional_rows[nr][a_field]
    if len(data_mentions) > 0:
       csvh.write_csv_data(data_mentions, 'pdf_mentions202110.csv')

In [ ]:
filter_mentions = {}
for dm in data_mentions:
    if data_mentions[dm]['action'] in ['add', 'reviewed']:
        filter_mentions[dm]={}
        for a_field in dm_headers:
            filter_mentions[dm][a_field] = data_mentions[dm][a_field]
print('filtered mentions:', len(filter_mentions))

In [ ]:
new_do_id_list =[]
for fm in filter_mentions:
    art_id = int(filter_mentions[fm]["id"])
    if not art_id in new_do_id_list:
        new_do_id_list.append(art_id)

# currend app DB
ukchapp_db = "db_files/app_db20211005.sqlite3"

no_data_pubs = pr_fns.get_pub_app_no_data(ukchapp_db)
#print(len(ids_w_data))
print(len(no_data_pubs))
print(new_do_id_list, len(new_do_id_list))
filter_mentions


int_idx = 0
revised_list = {}
if Path("./html_revised202111.csv").is_file():
    revised_list, rl_headers = csvh.get_csv_data('html_revised202111.csv')
    int_idx = len(revised_list)
    
already_revised =[]
for fm in revised_list:
    art_id = int(revised_list[fm]["id"])
    if not art_id in already_revised:
        already_revised.append(art_id)
    
for ndp in no_data_pubs:
    if not ndp[0] in new_do_id_list and ndp[0] > 616 and not ndp[0] in already_revised:
        int_idx += 1
        pub_id = ndp[0]
        pub_title = ndp[1]
        pub_doi = ndp[2]
        pub_url = ndp[3]
        data_record = {'id':pub_id, 'doi':pub_doi, 'title':pub_title} 
        print ('id',pub_id, '\n', pub_title)
        decide_action = False
        terminate = False
        while not decide_action:
            print('Action:')
            print(pub_url)
            print("https://doi.org/"+pub_doi)
            print('\ta) no data' )
            print('\ts) review')
            print('\td) next')
            print('\tf) terminate')
            print('\tSelect a, s, d, f:')
            lts = input()
            if lts == "a":
                data_record['action'] = 'no data'
                data_record['issue'] = "no data availability or supplementary data mentioned in html or pdf versions or article"
                revised_list[int_idx] = data_record
                decide_action = True
            if lts == "s":
                data_record['action'] = 'review'
                print ('issue:',data_mentions[dm]['issue'])
                add_this = input()
                data_record['issue'] = add_this
                revised_list[int_idx] = data_record
                decide_action = True
            if lts == "d":
                decide_action = True
            elif lts == 'f':
                decide_action = True
                terminate = True
        if terminate:
            break

if len(revised_list) > 0:
    csvh.write_csv_data(revised_list, 'html_revised202111.csv')

In [ ]:
if len(revised_list) > 0:
    csvh.write_csv_data(revised_list, 'html_revised202111.csv')
revised_list

In [ ]:
# functions for ChemDataExtractor
# not used for mining data references (suplementary/raw) or to get pdf metadata
from chemdataextractor import Document

# A function for getting a list of files from the directory
# This will be modified to get the list from a csv file
def get_files_list (source_dir):
    i_counter = 0
    files_list = []
    for filepath in sorted(source_dir.glob('*.pdf')):
        i_counter += 1
        files_list.append(filepath)
    return files_list

def cde_read_pdfs(a_file):
    pdf_f = open(a_file, 'rb')
    doc = Document.from_file(pdf_f)
    return doc

def find_doi(element_text):
    cr_re_01 = '10.\d{4,9}/[-._;()/:A-Z0-9]+'
    compare = re.search(cr_re_01, element_text, re.IGNORECASE)
    if compare != None:
        return compare.group()
    return ""

def get_db_id(doi_value, db_name = "app_db.sqlite3"):
    db_conn = dbh.DataBaseAdapter(db_name)
    table = 'articles'   
    id_val = db_conn.get_value(table, "id", "doi", doi_value)
    db_conn.close()
    if id_val != None:
        return id_val[0]
    else:
        return 0

def get_db_title(doi_value, db_name = "app_db.sqlite3"):
    db_conn = dbh.DataBaseAdapter(db_name)
    table = 'articles'   
    id_val = db_conn.get_value(table, "title", "doi", doi_value)
    db_conn.close()
    if id_val != None:
        return id_val[0]
    else:
        return 0

def get_close_dois(str_name, db_name = "prev_search.sqlite3"):
    db_conn = dbh.DataBaseAdapter(db_name)
    search_in = 'articles'
    fields_required = "id, doi, title, pdf_file"
    filter_str = "doi like '%"+str_name+"%';"

    db_titles = db_conn.get_values(search_in, fields_required, filter_str)
    db_conn.close()
    return db_titles

Get the name of the current app db file:

In [ ]:
# app db file with path: db_files/app_db.sqlite3
ukchapp_db = "db_files/app_db2.sqlite3"
while not Path(ukchapp_db).is_file():
    print('Please enter the name of app db file:')
    ukchapp_db = input()
ukchapp_db



In [ ]:
# get names and links for references in data mentions
data_mentions, dm_fields = csvh.get_csv_data('pdf_mentions_filtered_02.csv', 'num')

for dm in data_mentions:
    print("https://doi.org/" + data_mentions[dm]['doi'])
    ref_name = data_mentions[dm]['ref_name']
    while ref_name == "":
        print('Please enter the name of data object:')
        ref_name = input()
    ref_link = data_mentions[dm]['ref_link']
    while ref_link == "":
        print('Please enter the data object link:')
        ref_link = input()
    data_mentions[dm]['ref_name'] = ref_name
    data_mentions[dm]['ref_link'] = ref_link


In [ ]:
len(data_records)

In [ ]:
data_mentions

In [ ]:
from inspect import getmembers, isfunction

In [ ]:
help(pdfminer.high_level)

In [ ]:
!jupyter --version

In [ ]:
from platform import python_version
python_version()